# Creating synthetic SAMUeL data  data with SMOTE

## Description of SMOTE

SMOTE stands for Synthetic Minority Oversampling Technique [1]. SMOTE is more commonly used to create additional data to enhance modelling fitting, especially when one or more classes have low prevalence in the data set. Hence the description of *oversampling*. 

SMOTE works by finding near-neighbor points in the original data, and creating new data points from interpolating between two near-neighbor points.

Here we remove the real data used to create the synthetic data, leaving only the synthetic data. After generating synthetic data we remove any data points that, by chance, are identical to original real data points, and also remove 10% of points that are closest to the original data points. We measure 'closeness' by the Cartesian distance between standardised data values.

![](./images/smote.png)

*Demonstration of SMOTE method. (a) Data points with two features (shown on x and y axes) are represented. Points are colour-coded by class label. (b) A data point from a class is picked at random, shown by the black point, and then the closest neighbours of the same class are identified, as shown by yellow points. Here we show 3 closest neighbours, but the default in the SMOTE `Imbalanced-Learn` library is 6. One of those near-neighbour points is selected at random (shown by the second black point). A new data point, shown in red, is created at a random distance between the two selected data points.*

### Handling integer, binary, and categorical data

The standard SMOTE method generates floating point non-integer) values between data points. There are alternative ways of handing integer, binary, and categorical data using the SMOTE method. Here the methods we use are:

* *Integer* values: Round the resulting synthetic data point value to the closest integer.

* *Binary*: Code the value as 0 or 1, and round the resulting synthetic data point value to the closest integer (0 or 1).

* *Categorical*: One-hot encode the categorical feature. Generate the synthetic data for each category value. Identify the category with the highest value and set to 1 while setting all others to 0.

### Implementation with IMBLEARN

Here use the implementation in the IMBLEARN IMBALANCED-LEARN [2] 

[1] Chawla, N.V., Bowyer, K.W., Hall, L.O., Kegelmeyer, W.P. “SMOTE: Synthetic minority over-sampling technique,” Journal of Artificial Intelligence Research, vol. 16, pp. 321-357, 2002.

[2] Lemaitre, G., Nogueira, F. and Aridas, C. (2016), Imbalanced-learn: A Python Toolbox to Tackle the Curse of Imbalanced Datasets in Machine Learning. arXiv:1609.06570 (https://pypi.org/project/imbalanced-learn/, `pip install imbalanced-learn`).

## Load packages

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Import machine learning methods
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import auc
from sklearn.metrics import roc_curve
from sklearn.neighbors import NearestNeighbors

# Import package for SMOTE
import imblearn

# Turn warnings off to keep notebook clean
import warnings
warnings.filterwarnings("ignore")

## Load data

In [2]:
data_loc = './../data/sam_1/kfold_5fold/'

In [3]:
train_data, test_data, k_fold_synthetic_data = [], [], []

for i in range(5):
    
    train_data.append(pd.read_csv(data_loc + 'train_{0}.csv'.format(i)))
    test_data.append(pd.read_csv(data_loc + 'test_{0}.csv'.format(i)))
    k_fold_synthetic_data.append(pd.read_csv(data_loc + 'synth_train_{0}.csv'.format(i)))
    

## Functions

### Function to standardise data

In [4]:
def standardise_data(X_train, X_test):
    """
    Converts all data to a similar scale.
    Standardisation subtracts mean and divides by standard deviation
    for each feature.
    Standardised data will have a mena of 0 and standard deviation of 1.
    The training data mean and standard deviation is used to standardise both
    training and test set data.
    """
    
    # Initialise a new scaling object for normalising input data
    sc = StandardScaler() 

    # Set up the scaler just on the training set
    sc.fit(X_train)

    # Apply the scaler to the training and test sets
    train_std=sc.transform(X_train)
    test_std=sc.transform(X_test)
    
    return train_std, test_std

### Function to calculate accuracy measures

In [5]:
def calculate_accuracy(observed, predicted):
    
    """
    Calculates a range of accuracy scores from observed and predicted classes.
    
    Takes two list or NumPy arrays (observed class values, and predicted class 
    values), and returns a dictionary of results.
    
     1) observed positive rate: proportion of observed cases that are +ve
     2) Predicted positive rate: proportion of predicted cases that are +ve
     3) observed negative rate: proportion of observed cases that are -ve
     4) Predicted negative rate: proportion of predicted cases that are -ve  
     5) accuracy: proportion of predicted results that are correct    
     6) precision: proportion of predicted +ve that are correct
     7) recall: proportion of true +ve correctly identified
     8) f1: harmonic mean of precision and recall
     9) sensitivity: Same as recall
    10) specificity: Proportion of true -ve identified:        
    11) positive likelihood: increased probability of true +ve if test +ve
    12) negative likelihood: reduced probability of true +ve if test -ve
    13) false positive rate: proportion of false +ves in true -ve patients
    14) false negative rate: proportion of false -ves in true +ve patients
    15) true positive rate: Same as recall
    16) true negative rate
    17) positive predictive value: chance of true +ve if test +ve
    18) negative predictive value: chance of true -ve if test -ve
    
    """
    
    # Converts list to NumPy arrays
    if type(observed) == list:
        observed = np.array(observed)
    if type(predicted) == list:
        predicted = np.array(predicted)
    
    # Calculate accuracy scores
    observed_positives = observed == 1
    observed_negatives = observed == 0
    predicted_positives = predicted == 1
    predicted_negatives = predicted == 0
    
    true_positives = (predicted_positives == 1) & (observed_positives == 1)
    
    false_positives = (predicted_positives == 1) & (observed_positives == 0)
    
    true_negatives = (predicted_negatives == 1) & (observed_negatives == 1)
    
    false_negatives = (predicted_negatives == 1) & (observed_negatives == 0)
    
    accuracy = np.mean(predicted == observed)
    
    precision = (np.sum(true_positives) /
                 (np.sum(true_positives) + np.sum(false_positives)))
        
    recall = np.sum(true_positives) / np.sum(observed_positives)
    
    sensitivity = recall
    
    f1 = 2 * ((precision * recall) / (precision + recall))
    
    specificity = np.sum(true_negatives) / np.sum(observed_negatives)
    
    positive_likelihood = sensitivity / (1 - specificity)
    
    negative_likelihood = (1 - sensitivity) / specificity
    
    false_positive_rate = 1 - specificity
    
    false_negative_rate = 1 - sensitivity
    
    true_positive_rate = sensitivity
    
    true_negative_rate = specificity
    
    positive_predictive_value = (np.sum(true_positives) / 
                                 np.sum(observed_positives))
    
    negative_predictive_value = (np.sum(true_negatives) / 
                                  np.sum(observed_negatives))
    
    # Create dictionary for results, and add results
    results = dict()
    
    results['observed_positive_rate'] = np.mean(observed_positives)
    results['observed_negative_rate'] = np.mean(observed_negatives)
    results['predicted_positive_rate'] = np.mean(predicted_positives)
    results['predicted_negative_rate'] = np.mean(predicted_negatives)
    results['accuracy'] = accuracy
    results['precision'] = precision
    results['recall'] = recall
    results['f1'] = f1
    results['sensitivity'] = sensitivity
    results['specificity'] = specificity
    results['positive_likelihood'] = positive_likelihood
    results['negative_likelihood'] = negative_likelihood
    results['false_positive_rate'] = false_positive_rate
    results['false_negative_rate'] = false_negative_rate
    results['true_positive_rate'] = true_positive_rate
    results['true_negative_rate'] = true_negative_rate
    results['positive_predictive_value'] = positive_predictive_value
    results['negative_predictive_value'] = negative_predictive_value
    
    return results

### Function to create raw synthetic data

Data generated are floats; this will need processing for integer, binary, and categorical data.

In [6]:
def make_synthetic_data_smote(X, y, number_of_samples=[1000,1000]):
    """
    Synthetic data generation for two classes.
        
    Inputs
    ------
    original_data: X, y numpy arrays (y should have label 0 and 1)
    number_of_samples: number of samples to generate (list for y=0, y=1)
    
    Returns
    -------
    X_synthetic: NumPy array
    y_synthetic: NumPy array
    """
    
    from imblearn.over_sampling import SMOTE
    
    # Count instances in each class
    count_label_0 = np.sum(y==0)
    count_label_1 = np.sum(y==1)
    
    # SMOTE requires final class counts; add current counts to required counts
    n_class_0 = number_of_samples[0] + count_label_0
    n_class_1 = number_of_samples[1] + count_label_1

    # Get SMOTE points
    X_resampled, y_resampled = SMOTE(
        sampling_strategy = {0:n_class_0, 1:n_class_1}).fit_resample(X, y)

    # Get just the additional (synethetic) data points
    X_synthetic = X_resampled[len(X):]
    y_synthetic = y_resampled[len(y):]
                                                                   
    return X_synthetic, y_synthetic

### Function to process one-hot categorical data

In [7]:
def make_one_hot(x):
    """
    Takes a list/array/series and returns 1 for highest value and 0 for all 
    others
    
    """
    # Get argmax
    highest = np.argmax(x)
    # Set all values to zero
    x *= 0.0
    # Set argmax to one
    x[highest] = 1.0
    
    return x

### Function to add nearest real data neighbour to synthetic data

Find nearest neighbour in the real data set (based on Cartesian distance of standardised data).Find nearest neighbour in the real data set (based on Cartesian distance of standardised data).

In [8]:
def find_distance_to_closest_real_data(
    X_actual_train, X_actual_test, X_synthetic):

    """
    Find nearest neighbour in the real data set (based on Cartesian distance of
    standardised data).
    """
    
    # Standardise data (based on real training data)
    X_train_std, X_synth_std = standardise_data(
        X_actual_train, X_synthetic)
    X_train_std, X_test_std = standardise_data(
        X_actual_train, X_actual_test)

    # Get all real X data (combine standardised training + test data)
    X_real_std = np.concatenate([X_train_std, X_test_std], axis=0)

    # Use ScitLearn neighbors.NearestNeighbors to get nearest neighbour    
    nn = NearestNeighbors(n_neighbors=1, algorithm='auto').fit(X_real_std)
    dists, idxs = nn.kneighbors(X_synth_std)

    # Store in dictionary
    nearest_neighbours = dict()
    nearest_neighbours['distances'] = list(dists.flatten())
    nearest_neighbours['ids'] = list(idxs.flatten())
    
    return nearest_neighbours

Function to remove identical or nearest neighbour points.

In [9]:
def remove_near_neighbours(df, frac_remove=0.0):
    """Remove identical or nearest neighbour points"""
    
    # Remove synethetic data points with identical real data points
    # (use distance of <0.001 as effectively identical)
    
    identical = df['nn_distance'] < 0.001
    mask = identical == False
    df = df[mask]
    
    # Remove close neighbours    
    df.sort_values('nn_distance', ascending=False, inplace=True)
    number_to_keep = int(len(df) * (1 - frac_remove))
    df = df.head(number_to_keep)
    df = df.sample(frac=1.0)
    
    return df 
    

## Set lists of categorical (one-hot coded) and integer fields

In [10]:
# One hot column lists

col_list = [
    'MoreEqual80y', 
    'S1Gender',
    'S1Ethnicity',
    'S1OnsetInHospital',
    'S1OnsetTimeType',
    'S1OnsetDateType',
    'S1ArriveByAmbulance',
    'S1AdmissionHour',
    'S1AdmissionDay',
    'S1AdmissionQuarter',
    'S1AdmissionYear',
    'CongestiveHeartFailure',
    'Hypertension',
    'AtrialFibrillation',
    'Diabetes',
    'StrokeTIA',
    'AFAntiplatelet',
    'AFAnticoagulent',
    'S2NewAFDiagnosis',
    'S2StrokeType',
    'S2TIAInLastMonth']

X_col_names = list(train_data[0])
one_hot_cols = []
for col in col_list:
    one_hot = [x for x in X_col_names if x[0:len(col)] == col]
    one_hot_cols.append(one_hot)
    
integer_cols = [
    'S1AgeOnArrival',
    'S2RankinBeforeStroke',
    'Loc',
    'LocQuestions',
    'LocCommands',
    'BestGaze',
    'Visual',
    'FacialPalsy',
    'MotorArmLeft',
    'MotorArmRight',
    'MotorLegLeft',
    'MotorLegRight',
    'LimbAtaxia',
    'Sensory',
    'BestLanguage',
    'Dysarthria',
    'ExtinctionInattention',
    'S2NihssArrival']

# Get min and max for integers (will be used to clip synthetic data)

integer_min_max = dict()
for col in integer_cols:
    col_min = int(train_data[0][col].min())
    col_max = int(train_data[0][col].max())
    integer_min_max[col] = (col_min, col_max)
    
# Manually clip age to 30 - 100 to avoid using extremes
integer_min_max['S1AgeOnArrival'] = (30, 100)

## Create synthetic data

### Create synthetic data and process categorical (one-hot) and integer data

### Add distance measure to original data

In [11]:
# Set up list of K-fold synthetic data
k_fold_synthetic_data_with_distance = []

# Loop through k-folds
num_k_folds=5
for k_fold in range(num_k_folds):
       
    X_train = train_data[k_fold].drop(['S2Thrombolysis'], axis=1)
    X_test = test_data[k_fold].drop(['S2Thrombolysis'], axis=1)
    X_y_synthetic = k_fold_synthetic_data[k_fold]
    
    # Loop through stroke teams (calculate distance to patients in same unit)
    groups = X_train.groupby('StrokeTeam')
    
    # Set up list for synthetic data for each stroke team
    synthetic_dfs = []
    
    for index, group_df in groups:
        
        # Get training, test, and synethetic data for each k-fold
        actual_train = group_df.drop(['StrokeTeam'], axis=1)
        mask = X_test['StrokeTeam'] == index
        actual_test = X_test[mask].drop(['StrokeTeam'], axis=1)
        mask = X_y_synthetic['StrokeTeam'] == index
        synthetic_X = X_y_synthetic[mask].drop(
            ['StrokeTeam', 'S2Thrombolysis'], axis=1)
        synthetic_Y = X_y_synthetic[mask]['S2Thrombolysis']
        
        # Get nearest neighbour distance and ID
        nearest_neighbours = find_distance_to_closest_real_data(
            actual_train, actual_test, synthetic_X)
        
        # Store data for unit in a dataframe
        df = X_y_synthetic[mask]
        df['nn_distance'] = nearest_neighbours['distances']
        df['nn_id'] = nearest_neighbours['ids']
        
        # Append to list of unit dataframes
        synthetic_dfs.append(df)
        
    # Concatenate results, shuffle and store
    synthetic_data = pd.concat(synthetic_dfs)
    synthetic_data = synthetic_data.sample(frac=1.0)
    k_fold_synthetic_data_with_distance.append(synthetic_data)  

### Remove points with identical points or near neighbours in real data

Remove points with identical points or near neighbours in real data (these are examined by unit - two identical patients may exist if they are associated with different stroke units).~

In [12]:
k_fold_synthetic_data_restricted = []

for k_fold in range(num_k_folds):
    
    # Remove points with identical points or near neighbours in real data 
    restricted_data = remove_near_neighbours(
        k_fold_synthetic_data_with_distance[k_fold], 0.1)
    
    # Sample from synthetic data to get same size/balance as the original data
    original_data = pd.concat([train_data[k_fold], test_data[k_fold]], axis=0)                               
    num_class_0 = np.sum(original_data['S2Thrombolysis'] == 0)
    num_class_1 = np.sum(original_data['S2Thrombolysis'] == 1)
    
    mask = restricted_data['S2Thrombolysis'] == 0
    synth_class_0 = restricted_data[mask].sample(num_class_0, replace=True)
    mask = restricted_data['S2Thrombolysis'] == 1
    synth_class_1 = restricted_data[mask].sample(num_class_1, replace=True)
    
    # Reconstruct into single dataframe and shuffle
    df = pd.concat([synth_class_0, synth_class_1], axis=0)
    df = df.sample(frac=1.0)
    
    k_fold_synthetic_data_restricted.append(df)

### Save synthetic data

In [13]:
cols = list(train_data[0])
for k_fold in range(num_k_folds):
    # Save complete data
    data = k_fold_synthetic_data_restricted[k_fold]
    name = data_loc + f'synthetic_with_nn_2_{k_fold}.csv'
    data.to_csv(name, index=False)
    # Save restricted data without nearest neighbour info
    data = k_fold_synthetic_data_restricted[k_fold]
    data = data[cols]
    name = data_loc + f'synthetic_2_{k_fold}.csv'
    data.to_csv(name, index=False)